In [2]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

Using device: cuda


In [3]:
def train_val(model, criterion, optimizer, train_loader, val_loader, device, scheduler = None, use_scheduler = True):
    model.train()
    train_loss = 0
    correct = 0
    count = 0
    total = 0
    for i, data in enumerate(train_loader, 0):
        image, label = data
        image = image.to(device)
        label = label.to(device)
    
        optimizer.zero_grad()
        output = model(image)
        loss = criterion(output, label)

        train_loss += loss.item()

        pred = torch.max(output.data, 1)[1]
        cur_correct = (pred == label).sum().item()
        cur_loss = loss.item()

        loss.backward()

        optimizer.step()
        
        total += label.size(0)
        correct += cur_correct
        train_loss += cur_loss

    train_accuracy = correct/total
    train_loss = train_loss/len(train_loader)
    
    model.eval()
    valid_loss = 0
    correct = 0
    count = 0
    total = 0
    for i, data in enumerate(val_loader, 0):
        image, label = data
        image = image.to(device)
        label = label.to(device)
                
        output = model(image)
        loss = criterion(output, label)

        pred = torch.max(output.data, 1)[1]
        cur_correct = (pred == label).sum().item()
        cur_loss = loss.item()
            
        total += label.size(0)
        correct += cur_correct
        valid_loss += cur_loss

    valid_accuracy = correct/total
    valid_loss = valid_loss/len(val_loader)
    
    if use_scheduler:
        scheduler.step(valid_accuracy)

    return train_loss, train_accuracy, valid_loss, valid_accuracy

def test(model, criterion, dataloader, device):
    model.eval()
    test_loss = 0
    correct = 0
    count = 0
    total = 0
    for i, data in enumerate(dataloader, 0):
        image, label = data
        image = image.to(device)
        label = label.to(device)
                
        output = model(image)
        loss = criterion(output, label)

        pred = torch.max(output.data, 1)[1]
        cur_correct = (pred == label).sum().item()
        cur_loss = loss.item()
            
        total += label.size(0)
        correct += cur_correct
        test_loss += cur_loss

    accuracy = correct/total
    test_loss = test_loss/len(dataloader)

    return test_loss, accuracy

In [4]:
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(
            in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

In [5]:
class ZigZag_ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ZigZag_ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 128, num_blocks[3], stride=2)
        self.layer5 = self._make_layer(block, 64, num_blocks[4], stride=2)
        self.layer6 = self._make_layer(block, 128, num_blocks[5], stride=2)
        self.layer7 = self._make_layer(block, 256, num_blocks[6], stride=2)
        self.linear = nn.Linear(256*block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.layer5(out)
        out = self.layer6(out)
        out = self.layer7(out)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

zz_model = ZigZag_ResNet(BasicBlock, [2, 2, 2, 2, 2, 1, 1])
num_params = sum(p.numel() for p in zz_model.parameters() if p.requires_grad)
print(f"Num Params: {num_params}\n")

Num Params: 4891338



In [43]:
class Deep_Narrow_ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(Deep_Narrow_ResNet, self).__init__()
        self.in_planes = 32

        self.conv1 = nn.Conv2d(3, 32, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(32)
        self.layer1 = self._make_layer(block, 32, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 64, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 128, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.linear = nn.Linear(256*block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out
    
dn_model = Deep_Narrow_ResNet(BasicBlock, [13, 9, 3, 5])
num_params = sum(p.numel() for p in dn_model.parameters() if p.requires_grad)
print(f"Num Params: {num_params}\n")

Num Params: 4995754



In [5]:
import wandb
import os
os.environ['WANDB_API_KEY'] = '7af926a637567fde060a1a93e656d8887056249f'
wandb.login()

wandb: Currently logged in as: sj3828 (nyu-tandon). Use `wandb login --relogin` to force relogin


True

In [6]:
sweep_config = {
    "name": "hyperparameter_sweep",
    'metric': {'goal': 'maximize', 'name': 'test_accuracy'},
    "method": "grid",
    "parameters": {
        "factor": {
            "values": [0.1, 0.2, 0.3, 0.4, 0.5, 0.6]
        },
        "patience": {
            "values": [0, 1, 2, 3, 5, 10]
        }
    }
}

In [8]:
batch_size = 32
    
transform_train = torchvision.transforms.Compose([
  torchvision.transforms.RandomCrop(32, padding=4),
  torchvision.transforms.RandomHorizontalFlip(),
  torchvision.transforms.RandomResizedCrop(32, scale=(0.8, 1.0), ratio=(0.8, 1.2)),
  torchvision.transforms.ToTensor(),
  torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

transform_test = torchvision.transforms.Compose([
                  torchvision.transforms.ToTensor(), 
                  torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])

trainset = torchvision.datasets.CIFAR10(root = './data', train = True, download = True, transform = transform_train)

# Split the train data into train and validation sets
train_size = int(0.8 * len(trainset))
val_size = len(trainset) - train_size
trainset, valset = torch.utils.data.random_split(trainset, [train_size, val_size])

train_size = int(0.2 * len(trainset))
val_size = int(0.2 * len(valset))
trainset, _ = torch.utils.data.random_split(trainset, [train_size, len(trainset) - train_size])
valset, _ = torch.utils.data.random_split(valset, [val_size, len(valset) - val_size])

testset = torchvision.datasets.CIFAR10(root = './data', train = False, download = True, transform = transform_test)

train_loader = torch.utils.data.DataLoader(trainset, batch_size = batch_size, shuffle = True)
val_loader   = torch.utils.data.DataLoader(valset, batch_size = batch_size, shuffle = True)
test_loader  = torch.utils.data.DataLoader(testset, batch_size = batch_size, shuffle = False)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
    
def train_sweep():
    wandb.init()
    
    config = wandb.config
    config.factor = sweep_config['parameters']['factor']
    config.patience = sweep_config['parameters']['patience']
    
    best_test_acc = 0

    torch.cuda.empty_cache()
    model = ZigZag_ResNet(BasicBlock, [2, 2, 2, 2, 2, 1, 1])
    num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"Num Params: {num_params}\n")
    model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), 0.01, momentum = 0.8, weight_decay = 0.0005 , nesterov=True)

    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=config.factor, patience=config.patience, verbose = True)

    train_losses_ = []
    train_accuracies_ = []
    valid_losses_ = []
    valid_accuracies_ = []

    epochs = 50

    for epoch in range(epochs):
        print(f"\n\tEpoch: {epoch}")

        train_loss, train_accuracy, val_loss, val_accuracy = train_val(model, criterion, optimizer, 
                                                                    train_loader, val_loader, device,
                                                                    scheduler = scheduler, use_scheduler = True)
        train_losses_.append(train_loss)
        train_accuracies_.append(train_accuracy)
        valid_losses_.append(val_loss)
        valid_accuracies_.append(val_accuracy)
        print(f"\tTraining Loss: {round(train_loss, 4)}; Training Accuracy: {round(train_accuracy*100, 4)}%")
        print(f"\tValidation Loss: {round(val_loss, 4)}; Validation Accuracy: {round(val_accuracy*100, 4)}%")

    test_loss, test_accuracy = test(model, criterion, test_loader, device)
    print(f"\n\tTesting Loss: {round(test_loss, 4)}; Testing Accuracy: {round(test_accuracy*100, 4)}%")

    if test_accuracy > best_test_acc:
        best_test_acc = test_accuracy

        torch.save(model.state_dict(), 'zigzag_mini_resnet_7blocks.pth')

        metrics_dict = {'train_loss': train_losses_, 'train_accuracy': train_accuracies_, 
                      'valid_loss': valid_losses_, 'valid_accuracy': valid_accuracies_,
                      'test_loss': test_loss, 'test_accuracy': test_accuracy}
    wandb.log({'test_accuracy': test_accuracy})
    wandb.finish()

Files already downloaded and verified
Files already downloaded and verified


In [9]:
print(f"Train: {len(trainset)}; Validation: {len(valset)}; Test: {len(testset)}")

Train: 8000; Validation: 2000; Test: 10000


In [10]:
sweep_id = wandb.sweep(sweep_config, project = "ZigZag-Mini-Resnet")
wandb.agent(sweep_id, train_sweep)

Create sweep with ID: qe6f7ssq
Sweep URL: https://wandb.ai/nyu-tandon/ZigZag-Mini-Resnet/sweeps/qe6f7ssq


wandb: Agent Starting Run: ijsoju5d with config:
wandb: 	factor: 0.1
wandb: 	patience: 0


wandb: WARNING Config item 'factor' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'patience' was locked by 'sweep' (ignored update).


Num Params: 4891338


	Epoch: 0
	Training Loss: 4.0732; Training Accuracy: 24.3875%
	Validation Loss: 1.8463; Validation Accuracy: 29.7%

	Epoch: 1
	Training Loss: 3.6255; Training Accuracy: 32.7375%
	Validation Loss: 1.7272; Validation Accuracy: 35.6%

	Epoch: 2
Epoch 00003: reducing learning rate of group 0 to 1.0000e-03.
	Training Loss: 3.4086; Training Accuracy: 36.7625%
	Validation Loss: 1.7302; Validation Accuracy: 34.75%

	Epoch: 3
	Training Loss: 3.1282; Training Accuracy: 42.55%
	Validation Loss: 1.5205; Validation Accuracy: 42.85%

	Epoch: 4
	Training Loss: 3.0019; Training Accuracy: 44.8%
	Validation Loss: 1.4858; Validation Accuracy: 45.2%

	Epoch: 5
	Training Loss: 2.9302; Training Accuracy: 46.5875%
	Validation Loss: 1.4341; Validation Accuracy: 48.15%

	Epoch: 6
Epoch 00007: reducing learning rate of group 0 to 1.0000e-04.
	Training Loss: 2.8656; Training Accuracy: 47.5875%
	Validation Loss: 1.4234; Validation Accuracy: 47.65%

	Epoch: 7
	Training Loss: 2.8217; Training 

test_accuracy,▁
test_accuracy,0.516


...

wandb: Agent Starting Run: xe8742pd with config:
wandb: 	factor: 0.6
wandb: 	patience: 10


wandb: WARNING Config item 'factor' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'patience' was locked by 'sweep' (ignored update).


Num Params: 4891338


	Epoch: 0
	Training Loss: 4.1692; Training Accuracy: 21.6625%
	Validation Loss: 1.9251; Validation Accuracy: 27.05%

	Epoch: 1
	Training Loss: 3.6585; Training Accuracy: 31.675%
	Validation Loss: 1.753; Validation Accuracy: 36.4%

	Epoch: 2
	Training Loss: 3.4166; Training Accuracy: 35.9625%
	Validation Loss: 1.8182; Validation Accuracy: 37.1%

	Epoch: 3
	Training Loss: 3.2744; Training Accuracy: 39.9875%
	Validation Loss: 1.625; Validation Accuracy: 40.5%

	Epoch: 4
	Training Loss: 3.1081; Training Accuracy: 42.775%
	Validation Loss: 1.5743; Validation Accuracy: 41.7%

	Epoch: 5
	Training Loss: 2.9979; Training Accuracy: 44.925%
	Validation Loss: 1.5765; Validation Accuracy: 42.75%

	Epoch: 6
	Training Loss: 2.9191; Training Accuracy: 47.125%
	Validation Loss: 1.4574; Validation Accuracy: 47.45%

	Epoch: 7
	Training Loss: 2.8178; Training Accuracy: 49.2625%
	Validation Loss: 1.4552; Validation Accuracy: 48.05%

	Epoch: 8
	Training Loss: 2.7083; Training Accuracy:

test_accuracy,▁
test_accuracy,0.7723


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.


### Running the best hyperparameters for 100 epochs

In [12]:
batch_size = 32
    
transform_train = torchvision.transforms.Compose([
  torchvision.transforms.RandomCrop(32, padding=4),
  torchvision.transforms.RandomHorizontalFlip(),
  torchvision.transforms.RandomResizedCrop(32, scale=(0.8, 1.0), ratio=(0.8, 1.2)),
  torchvision.transforms.ToTensor(),
  torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

transform_test = torchvision.transforms.Compose([
                  torchvision.transforms.ToTensor(), 
                  torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])

trainset = torchvision.datasets.CIFAR10(root = './data', train = True, download = True, transform = transform_train)

# Split the train data into train and validation sets
train_size = int(0.8 * len(trainset))
val_size = len(trainset) - train_size
trainset, valset = torch.utils.data.random_split(trainset, [train_size, val_size])

testset = torchvision.datasets.CIFAR10(root = './data', train = False, download = True, transform = transform_test)

train_loader = torch.utils.data.DataLoader(trainset, batch_size = batch_size, shuffle = True)
val_loader   = torch.utils.data.DataLoader(valset, batch_size = batch_size, shuffle = True)
test_loader  = torch.utils.data.DataLoader(testset, batch_size = batch_size, shuffle = False)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
    
best_test_acc = 0

torch.cuda.empty_cache()
model = ZigZag_ResNet(BasicBlock, [2, 2, 2, 2, 2, 1, 1])
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Num Params: {num_params}\n")
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), 0.01, momentum = 0.8, weight_decay = 0.0005 , nesterov=True)

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.1, patience=5, verbose = True)

train_losses_ = []
train_accuracies_ = []
valid_losses_ = []
valid_accuracies_ = []

Files already downloaded and verified
Files already downloaded and verified
Num Params: 4891338



In [13]:
print(f"Train: {len(trainset)}; Validation: {len(valset)}; Test: {len(testset)}")

Train: 40000; Validation: 10000; Test: 10000


In [14]:
epochs = 100

for epoch in range(epochs):
    print(f"\n\tEpoch: {epoch}")

    train_loss, train_accuracy, val_loss, val_accuracy = train_val(model, criterion, optimizer, 
                                                                train_loader, val_loader, device,
                                                                scheduler = scheduler, use_scheduler = True)
    train_losses_.append(train_loss)
    train_accuracies_.append(train_accuracy)
    valid_losses_.append(val_loss)
    valid_accuracies_.append(val_accuracy)
    print(f"\tTraining Loss: {round(train_loss, 4)}; Training Accuracy: {round(train_accuracy*100, 4)}%")
    print(f"\tValidation Loss: {round(val_loss, 4)}; Validation Accuracy: {round(val_accuracy*100, 4)}%")

test_loss, test_accuracy = test(model, criterion, test_loader, device)
print(f"\n\tTesting Loss: {round(test_loss, 4)}; Testing Accuracy: {round(test_accuracy*100, 4)}%")

if test_accuracy > best_test_acc:
    best_test_acc = test_accuracy

    torch.save(model.state_dict(), 'zigzag_resnet_tuned.pth')

    metrics_dict = {'train_loss': train_losses_, 'train_accuracy': train_accuracies_, 
                  'valid_loss': valid_losses_, 'valid_accuracy': valid_accuracies_,
                  'test_loss': test_loss, 'test_accuracy': test_accuracy}


	Epoch: 0
	Training Loss: 3.4809; Training Accuracy: 35.82%
	Validation Loss: 1.4469; Validation Accuracy: 47.46%

	Epoch: 1
	Training Loss: 2.7637; Training Accuracy: 50.04%
	Validation Loss: 1.2508; Validation Accuracy: 54.9%

	Epoch: 2
	Training Loss: 2.2853; Training Accuracy: 59.295%
	Validation Loss: 1.0214; Validation Accuracy: 63.82%

	Epoch: 3
	Training Loss: 1.9332; Training Accuracy: 66.0725%
	Validation Loss: 0.8924; Validation Accuracy: 68.26%

	Epoch: 4
	Training Loss: 1.707; Training Accuracy: 70.775%
	Validation Loss: 0.7887; Validation Accuracy: 72.32%

	Epoch: 5
	Training Loss: 1.5247; Training Accuracy: 73.725%
	Validation Loss: 0.7433; Validation Accuracy: 74.25%

	Epoch: 6
	Training Loss: 1.3991; Training Accuracy: 75.625%
	Validation Loss: 0.694; Validation Accuracy: 76.01%

	Epoch: 7
	Training Loss: 1.2909; Training Accuracy: 77.875%
	Validation Loss: 0.7053; Validation Accuracy: 76.1%

	Epoch: 8
	Training Loss: 1.2107; Training Accuracy: 79.3275%
	Validation Lo

### ZigZag Scheduler

In [6]:
import numpy as np

class ZigZagLROnPlateau(torch.optim.lr_scheduler._LRScheduler):
    def __init__(self, optimizer, mode='min', up_factor=1.1, down_factor=0.8, up_patience=10, down_patience=10, verbose=True):
        super(ZigZagLROnPlateau).__init__()
        self.optimizer = optimizer
        self.mode = mode
        self.up_factor = 1 + up_factor
        self.down_factor = 1 - down_factor
        self.up_patience = up_patience
        self.down_patience = down_patience
        self.num_bad_epochs = 0
        self.num_good_epochs = 0
        self.best_metric = np.Inf if self.mode == 'min' else -np.Inf
        self.verbose = verbose

    def step(self, metric):
        if self.mode == 'min':
            if metric < self.best_metric:
                self.best_metric = metric
                self.num_bad_epochs = 0
                self.num_good_epochs += 1
                if self.num_good_epochs > self.up_patience:
                    old_lr = self.optimizer.param_groups[0]['lr']
                    new_lr = old_lr * self.up_factor
                    self.optimizer.param_groups[0]['lr'] = new_lr
                    if self.verbose:
                        print(f"increasing learning rate of group 0 to {new_lr:.4e}.")
                    self.num_good_epochs = 0
            else:
                self.num_bad_epochs += 1
                self.num_good_epochs = 0
                if self.num_bad_epochs > self.down_patience:
                    old_lr = self.optimizer.param_groups[0]['lr']
                    new_lr = old_lr * self.down_factor
                    self.optimizer.param_groups[0]['lr'] = new_lr
                    if self.verbose:
                        print(f"reducing learning rate of group 0 to {new_lr:.4e}.")
                    self.num_bad_epochs = 0
        else:
            if metric > self.best_metric:
                self.best_metric = metric
                self.num_bad_epochs = 0
                self.num_good_epochs += 1
                if self.num_good_epochs > self.up_patience:
                    old_lr = self.optimizer.param_groups[0]['lr']
                    new_lr = old_lr * self.up_factor
                    self.optimizer.param_groups[0]['lr'] = new_lr
                    if self.verbose:
                        print(f"increasing learning rate of group 0 to {new_lr:.4e}.")
                    self.num_good_epochs = 0
            else:
                self.num_bad_epochs += 1
                self.num_good_epochs = 0
                if self.num_bad_epochs > self.down_patience:
                    old_lr = self.optimizer.param_groups[0]['lr']
                    new_lr = old_lr * self.down_factor
                    self.optimizer.param_groups[0]['lr'] = new_lr
                    if self.verbose:
                        print(f"reducing learning rate of group 0 to {new_lr:.4e}.")
                    self.num_bad_epochs = 0

In [21]:
batch_size = 32
    
transform_train = torchvision.transforms.Compose([
  torchvision.transforms.RandomCrop(32, padding=4),
  torchvision.transforms.RandomHorizontalFlip(),
  torchvision.transforms.RandomResizedCrop(32, scale=(0.8, 1.0), ratio=(0.8, 1.2)),
  torchvision.transforms.ToTensor(),
  torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

transform_test = torchvision.transforms.Compose([
                  torchvision.transforms.ToTensor(), 
                  torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])

trainset = torchvision.datasets.CIFAR10(root = './data', train = True, download = True, transform = transform_train)

# Split the train data into train and validation sets
train_size = int(0.8 * len(trainset))
val_size = len(trainset) - train_size
trainset, valset = torch.utils.data.random_split(trainset, [train_size, val_size])

testset = torchvision.datasets.CIFAR10(root = './data', train = False, download = True, transform = transform_test)

train_loader = torch.utils.data.DataLoader(trainset, batch_size = batch_size, shuffle = True)
val_loader   = torch.utils.data.DataLoader(valset, batch_size = batch_size, shuffle = True)
test_loader  = torch.utils.data.DataLoader(testset, batch_size = batch_size, shuffle = False)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
    
best_test_acc = 0

torch.cuda.empty_cache()
model = ZigZag_ResNet(BasicBlock, [2, 2, 2, 2, 2, 1, 1])
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Num Params: {num_params}\n")
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), 0.01, momentum = 0.8, weight_decay = 0.0005 , nesterov=True)

scheduler = ZigZagLROnPlateau(optimizer, mode='max', up_factor=0.2, down_factor=0.2, up_patience=1, down_patience=1, verbose = True)

train_losses_ = []
train_accuracies_ = []
valid_losses_ = []
valid_accuracies_ = []

Files already downloaded and verified
Files already downloaded and verified
Num Params: 4891338



In [22]:
epochs = 20

for epoch in range(epochs):
    print(f"\n\tEpoch: {epoch}")

    train_loss, train_accuracy, val_loss, val_accuracy = train_val(model, criterion, optimizer, 
                                                                train_loader, val_loader, device,
                                                                scheduler = scheduler, use_scheduler = True)
    train_losses_.append(train_loss)
    train_accuracies_.append(train_accuracy)
    valid_losses_.append(val_loss)
    valid_accuracies_.append(val_accuracy)
    print(f"\tTraining Loss: {round(train_loss, 4)}; Training Accuracy: {round(train_accuracy*100, 4)}%")
    print(f"\tValidation Loss: {round(val_loss, 4)}; Validation Accuracy: {round(val_accuracy*100, 4)}%")

test_loss, test_accuracy = test(model, criterion, test_loader, device)
print(f"\n\tTesting Loss: {round(test_loss, 4)}; Testing Accuracy: {round(test_accuracy*100, 4)}%")

if test_accuracy > best_test_acc:
    best_test_acc = test_accuracy

    metrics_dict = {'train_loss': train_losses_, 'train_accuracy': train_accuracies_, 
                  'valid_loss': valid_losses_, 'valid_accuracy': valid_accuracies_,
                  'test_loss': test_loss, 'test_accuracy': test_accuracy}


	Epoch: 0
	Training Loss: 3.57; Training Accuracy: 33.68%
	Validation Loss: 1.5733; Validation Accuracy: 42.35%

	Epoch: 1
increasing learning rate of group 0 to 1.2000e-02.
	Training Loss: 2.8566; Training Accuracy: 48.1%
	Validation Loss: 1.286; Validation Accuracy: 53.05%

	Epoch: 2
	Training Loss: 2.398; Training Accuracy: 57.4025%
	Validation Loss: 1.0792; Validation Accuracy: 61.64%

	Epoch: 3
increasing learning rate of group 0 to 1.4400e-02.
	Training Loss: 1.978; Training Accuracy: 65.3825%
	Validation Loss: 0.9306; Validation Accuracy: 67.73%

	Epoch: 4
	Training Loss: 1.8; Training Accuracy: 68.72%
	Validation Loss: 0.9533; Validation Accuracy: 66.83%

	Epoch: 5
	Training Loss: 1.5869; Training Accuracy: 72.605%
	Validation Loss: 0.8102; Validation Accuracy: 72.1%

	Epoch: 6
increasing learning rate of group 0 to 1.7280e-02.
	Training Loss: 1.4401; Training Accuracy: 75.4325%
	Validation Loss: 0.7403; Validation Accuracy: 74.69%

	Epoch: 7
	Training Loss: 1.3941; Training A

### ZigZag Sweep

In [7]:
import wandb
import os
os.environ['WANDB_API_KEY'] = '7af926a637567fde060a1a93e656d8887056249f'
wandb.login()

wandb: Currently logged in as: sj3828 (nyu-tandon). Use `wandb login --relogin` to force relogin


True

In [8]:
sweep_config = {
    "name": "hyperparameter_sweep",
    'metric': {'goal': 'maximize', 'name': 'test_accuracy'},
    "method": "grid",
    "parameters": {
        "up_factor": {
            "values": [0.1, 0.3, 0.5]
        },
        "down_factor": {
            "values": [0.1, 0.3, 0.5]
        },
        "up_patience": {
            "values": [1, 3, 5]
        },
        "down_patience": {
            "values": [1, 3, 5]
        }
    }
}

In [9]:
batch_size = 32
    
transform_train = torchvision.transforms.Compose([
  torchvision.transforms.RandomCrop(32, padding=4),
  torchvision.transforms.RandomHorizontalFlip(),
  torchvision.transforms.RandomResizedCrop(32, scale=(0.8, 1.0), ratio=(0.8, 1.2)),
  torchvision.transforms.ToTensor(),
  torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

transform_test = torchvision.transforms.Compose([
                  torchvision.transforms.ToTensor(), 
                  torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])

trainset = torchvision.datasets.CIFAR10(root = './data', train = True, download = True, transform = transform_train)

# Split the train data into train and validation sets
train_size = int(0.8 * len(trainset))
val_size = len(trainset) - train_size
trainset, valset = torch.utils.data.random_split(trainset, [train_size, val_size])

train_size = int(0.2 * len(trainset))
val_size = int(0.2 * len(valset))
trainset, _ = torch.utils.data.random_split(trainset, [train_size, len(trainset) - train_size])
valset, _ = torch.utils.data.random_split(valset, [val_size, len(valset) - val_size])

testset = torchvision.datasets.CIFAR10(root = './data', train = False, download = True, transform = transform_test)

train_loader = torch.utils.data.DataLoader(trainset, batch_size = batch_size, shuffle = True)
val_loader   = torch.utils.data.DataLoader(valset, batch_size = batch_size, shuffle = True)
test_loader  = torch.utils.data.DataLoader(testset, batch_size = batch_size, shuffle = False)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

def train_sweep():
    wandb.init()
    
    config = wandb.config
    config.up_factor = sweep_config['parameters']['up_factor']
    config.down_factor = sweep_config['parameters']['down_factor']
    config.up_patience = sweep_config['parameters']['up_patience']
    config.down_patience = sweep_config['parameters']['down_patience']
    
    torch.cuda.empty_cache()
    model = ZigZag_ResNet(BasicBlock, [2, 2, 2, 2, 2, 1, 1])
    num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"Num Params: {num_params}\n")
    model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), 0.01, momentum = 0.8, weight_decay = 0.0005 , nesterov=True)

    scheduler = ZigZagLROnPlateau(optimizer, mode='max', up_factor=config.up_factor, down_factor=config.down_factor, 
                                                         up_patience=config.up_patience, down_patience=config.down_patience, 
                                                         verbose = True)


    train_losses_ = []
    train_accuracies_ = []
    valid_losses_ = []
    valid_accuracies_ = []

    epochs = 25

    for epoch in range(epochs):
        print(f"\n\tEpoch: {epoch}")

        train_loss, train_accuracy, val_loss, val_accuracy = train_val(model, criterion, optimizer, 
                                                                    train_loader, val_loader, device,
                                                                    scheduler = scheduler, use_scheduler = True)
        train_losses_.append(train_loss)
        train_accuracies_.append(train_accuracy)
        valid_losses_.append(val_loss)
        valid_accuracies_.append(val_accuracy)
        print(f"\tTraining Loss: {round(train_loss, 4)}; Training Accuracy: {round(train_accuracy*100, 4)}%")
        print(f"\tValidation Loss: {round(val_loss, 4)}; Validation Accuracy: {round(val_accuracy*100, 4)}%")

    test_loss, test_accuracy = test(model, criterion, test_loader, device)
    print(f"\n\tTesting Loss: {round(test_loss, 4)}; Testing Accuracy: {round(test_accuracy*100, 4)}%")

    wandb.log({'test_accuracy': test_accuracy})
    wandb.finish()

Files already downloaded and verified
Files already downloaded and verified


In [10]:
print(f"Train: {len(trainset)}; Validation: {len(valset)}; Test: {len(testset)}")

Train: 8000; Validation: 2000; Test: 10000


In [11]:
sweep_id = wandb.sweep(sweep_config, project = "Mini-ZigZag__Resnet__ZigZag-Scheduler")
wandb.agent(sweep_id, train_sweep)

500 response executing GraphQL.
{"errors":[{"message":"An internal error occurred. Please contact support.","path":["upsertSweep"]}],"data":{"upsertSweep":null}}
wandb: ERROR Error while calling W&B API: An internal error occurred. Please contact support. (<Response [500]>)


Create sweep with ID: nun2d9tx
Sweep URL: https://wandb.ai/nyu-tandon/Mini-ZigZag__Resnet__ZigZag-Scheduler/sweeps/nun2d9tx


wandb: Agent Starting Run: fwtmuknh with config:
wandb: 	down_factor: 0.1
wandb: 	down_patience: 1
wandb: 	up_factor: 0.1
wandb: 	up_patience: 1


wandb: WARNING Config item 'up_factor' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'down_factor' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'up_patience' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'down_patience' was locked by 'sweep' (ignored update).


Num Params: 4891338


	Epoch: 0
	Training Loss: 4.0888; Training Accuracy: 24.1875%
	Validation Loss: 1.8659; Validation Accuracy: 31.65%

	Epoch: 1
increasing learning rate of group 0 to 1.1000e-02.
	Training Loss: 3.6231; Training Accuracy: 32.95%
	Validation Loss: 1.6921; Validation Accuracy: 36.5%

	Epoch: 2
	Training Loss: 3.4162; Training Accuracy: 36.375%
	Validation Loss: 1.7097; Validation Accuracy: 36.55%

	Epoch: 3
increasing learning rate of group 0 to 1.2100e-02.
	Training Loss: 3.2488; Training Accuracy: 40.8125%
	Validation Loss: 1.6039; Validation Accuracy: 40.9%

	Epoch: 4
	Training Loss: 3.1135; Training Accuracy: 42.6125%
	Validation Loss: 1.4722; Validation Accuracy: 46.4%

	Epoch: 5
	Training Loss: 2.9578; Training Accuracy: 45.325%
	Validation Loss: 1.5025; Validation Accuracy: 43.2%

	Epoch: 6
	Training Loss: 2.8272; Training Accuracy: 48.975%
	Validation Loss: 1.4111; Validation Accuracy: 48.4%

	Epoch: 7
increasing learning rate of group 0 to 1.3310e-02.
	Train

...

wandb: Agent Starting Run: kezulrdl with config:
wandb: 	down_factor: 0.5
wandb: 	down_patience: 5
wandb: 	up_factor: 0.5
wandb: 	up_patience: 5


wandb: WARNING Config item 'up_factor' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'down_factor' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'up_patience' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'down_patience' was locked by 'sweep' (ignored update).


Num Params: 4891338


	Epoch: 0
	Training Loss: 4.0661; Training Accuracy: 24.9%
	Validation Loss: 1.8491; Validation Accuracy: 31.05%

	Epoch: 1
	Training Loss: 3.6166; Training Accuracy: 32.8125%
	Validation Loss: 1.7679; Validation Accuracy: 34.45%

	Epoch: 2
	Training Loss: 3.3855; Training Accuracy: 37.9875%
	Validation Loss: 1.6479; Validation Accuracy: 39.9%

	Epoch: 3
	Training Loss: 3.2053; Training Accuracy: 41.225%
	Validation Loss: 1.7593; Validation Accuracy: 37.95%

	Epoch: 4
	Training Loss: 3.0631; Training Accuracy: 43.9875%
	Validation Loss: 1.6377; Validation Accuracy: 41.35%

	Epoch: 5
	Training Loss: 2.989; Training Accuracy: 45.5%
	Validation Loss: 1.4896; Validation Accuracy: 45.25%

	Epoch: 6
	Training Loss: 2.8438; Training Accuracy: 48.075%
	Validation Loss: 1.3917; Validation Accuracy: 48.55%

	Epoch: 7
	Training Loss: 2.7045; Training Accuracy: 50.85%
	Validation Loss: 1.3295; Validation Accuracy: 51.75%

	Epoch: 8
	Training Loss: 2.5884; Training Accuracy: 5

test_accuracy,▁
test_accuracy,0.6811


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.


### Best Parameters for 100 epochs

In [12]:
batch_size = 32
    
transform_train = torchvision.transforms.Compose([
  torchvision.transforms.RandomCrop(32, padding=4),
  torchvision.transforms.RandomHorizontalFlip(),
  torchvision.transforms.RandomResizedCrop(32, scale=(0.8, 1.0), ratio=(0.8, 1.2)),
  torchvision.transforms.ToTensor(),
  torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

transform_test = torchvision.transforms.Compose([
                  torchvision.transforms.ToTensor(), 
                  torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])

trainset = torchvision.datasets.CIFAR10(root = './data', train = True, download = True, transform = transform_train)

# Split the train data into train and validation sets
train_size = int(0.8 * len(trainset))
val_size = len(trainset) - train_size
trainset, valset = torch.utils.data.random_split(trainset, [train_size, val_size])

testset = torchvision.datasets.CIFAR10(root = './data', train = False, download = True, transform = transform_test)

train_loader = torch.utils.data.DataLoader(trainset, batch_size = batch_size, shuffle = True)
val_loader   = torch.utils.data.DataLoader(valset, batch_size = batch_size, shuffle = True)
test_loader  = torch.utils.data.DataLoader(testset, batch_size = batch_size, shuffle = False)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
    
best_test_acc = 0

torch.cuda.empty_cache()
model = ZigZag_ResNet(BasicBlock, [2, 2, 2, 2, 2, 1, 1])
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Num Params: {num_params}\n")
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), 0.01, momentum = 0.8, weight_decay = 0.0005 , nesterov=True)

scheduler = ZigZagLROnPlateau(optimizer, mode='max', up_factor=0.3, down_factor=0.5, 
                                                     up_patience=1, down_patience=1, 
                                                     verbose = True)
train_losses_ = []
train_accuracies_ = []
valid_losses_ = []
valid_accuracies_ = []

Files already downloaded and verified
Files already downloaded and verified
Num Params: 4891338



In [13]:
epochs = 100

for epoch in range(epochs):
    print(f"\n\tEpoch: {epoch}")

    train_loss, train_accuracy, val_loss, val_accuracy = train_val(model, criterion, optimizer, 
                                                                train_loader, val_loader, device,
                                                                scheduler = scheduler, use_scheduler = True)
    train_losses_.append(train_loss)
    train_accuracies_.append(train_accuracy)
    valid_losses_.append(val_loss)
    valid_accuracies_.append(val_accuracy)
    print(f"\tTraining Loss: {round(train_loss, 4)}; Training Accuracy: {round(train_accuracy*100, 4)}%")
    print(f"\tValidation Loss: {round(val_loss, 4)}; Validation Accuracy: {round(val_accuracy*100, 4)}%")

test_loss, test_accuracy = test(model, criterion, test_loader, device)
print(f"\n\tTesting Loss: {round(test_loss, 4)}; Testing Accuracy: {round(test_accuracy*100, 4)}%")

if test_accuracy > best_test_acc:
    best_test_acc = test_accuracy

    metrics_dict = {'train_loss': train_losses_, 'train_accuracy': train_accuracies_, 
                  'valid_loss': valid_losses_, 'valid_accuracy': valid_accuracies_,
                  'test_loss': test_loss, 'test_accuracy': test_accuracy}


	Epoch: 0
	Training Loss: 3.4926; Training Accuracy: 35.61%
	Validation Loss: 1.5039; Validation Accuracy: 44.86%

	Epoch: 1
increasing learning rate of group 0 to 1.3000e-02.
	Training Loss: 2.817; Training Accuracy: 49.03%
	Validation Loss: 1.2185; Validation Accuracy: 56.55%

	Epoch: 2
	Training Loss: 2.3721; Training Accuracy: 57.6475%
	Validation Loss: 1.0677; Validation Accuracy: 62.27%

	Epoch: 3
increasing learning rate of group 0 to 1.6900e-02.
	Training Loss: 1.9618; Training Accuracy: 65.5025%
	Validation Loss: 0.9125; Validation Accuracy: 68.24%

	Epoch: 4
	Training Loss: 1.7833; Training Accuracy: 69.14%
	Validation Loss: 0.9361; Validation Accuracy: 67.61%

	Epoch: 5
	Training Loss: 1.5898; Training Accuracy: 72.58%
	Validation Loss: 0.8057; Validation Accuracy: 71.9%

	Epoch: 6
increasing learning rate of group 0 to 2.1970e-02.
	Training Loss: 1.4362; Training Accuracy: 75.5475%
	Validation Loss: 0.7226; Validation Accuracy: 75.22%

	Epoch: 7
	Training Loss: 1.4233; Tra

### Restarts

In [30]:
import numpy as np

class ZigZagLROnPlateauRestarts(torch.optim.lr_scheduler._LRScheduler):
    def __init__(self, optimizer, mode='min', lr=0.01, up_factor=1.1, down_factor=0.8, up_patience=10, down_patience=10, restart_after=30, verbose=True):
        super(ZigZagLROnPlateauRestarts).__init__()
        self.optimizer = optimizer
        self.mode = mode
        self.up_factor = 1 + up_factor
        self.down_factor = 1 - down_factor
        self.up_patience = up_patience
        self.down_patience = down_patience
        self.num_bad_epochs = 0
        self.num_good_epochs = 0
        self.best_metric = np.Inf if self.mode == 'min' else -np.Inf
        self.best_lr = lr
        self.restart_after = restart_after
        self.verbose = verbose
        self.num_epochs = 0

    def step(self, metric):
        self.num_epochs += 1
        if self.mode == 'min':
            if metric < self.best_metric:
                self.best_metric = metric
                self.best_lr = self.optimizer.param_groups[0]['lr']
                self.num_bad_epochs = 0
                self.num_good_epochs += 1
                if self.num_good_epochs > self.up_patience:
                    old_lr = self.optimizer.param_groups[0]['lr']
                    new_lr = old_lr * self.up_factor
                    self.optimizer.param_groups[0]['lr'] = new_lr
                    if self.verbose:
                        print(f"increasing learning rate of group 0 to {new_lr:.4e}.")
                    self.num_good_epochs = 0
            else:
                self.num_bad_epochs += 1
                self.num_good_epochs = 0
                if self.num_bad_epochs > self.down_patience:
                    old_lr = self.optimizer.param_groups[0]['lr']
                    new_lr = old_lr * self.down_factor
                    self.optimizer.param_groups[0]['lr'] = new_lr
                    if self.verbose:
                        print(f"reducing learning rate of group 0 to {new_lr:.4e}.")
                    self.num_bad_epochs = 0
        else:
            if metric > self.best_metric:
                self.best_metric = metric
                self.best_lr = self.optimizer.param_groups[0]['lr']
                self.num_bad_epochs = 0
                self.num_good_epochs += 1
                if self.num_good_epochs > self.up_patience:
                    old_lr = self.optimizer.param_groups[0]['lr']
                    new_lr = old_lr * self.up_factor
                    self.optimizer.param_groups[0]['lr'] = new_lr
                    if self.verbose:
                        print(f"increasing learning rate of group 0 to {new_lr:.4e}.")
                    self.num_good_epochs = 0
            else:
                self.num_bad_epochs += 1
                self.num_good_epochs = 0
                if self.num_bad_epochs > self.down_patience:
                    old_lr = self.optimizer.param_groups[0]['lr']
                    new_lr = old_lr * self.down_factor
                    self.optimizer.param_groups[0]['lr'] = new_lr
                    if self.verbose:
                        print(f"reducing learning rate of group 0 to {new_lr:.4e}.")
                    self.num_bad_epochs = 0
                    
        if self.num_epochs % self.restart_after == 0:
            self.best_metric = np.Inf if self.mode == 'min' else -np.Inf
            self.optimizer.param_groups[0]['lr'] = self.best_lr
            if self.verbose:
                print(f"restart: setting learning rate of group 0 to best learning rate value: {self.best_lr:.4e}.")

In [ ]:
# transform_train = torchvision.transforms.Compose([
#     torchvision.transforms.RandomCrop(32, padding=4),
#     torchvision.transforms.RandomHorizontalFlip(),
#     torchvision.transforms.RandomRotation(15),
#     torchvision.transforms.RandomResizedCrop(32, scale=(0.8, 1.0), ratio=(0.8, 1.2)),
#     torchvision.transforms.RandomAffine(degrees=15, translate=(0.1, 0.1), scale=(0.8, 1.2)),
#     torchvision.transforms.ToTensor(),
#     torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
#     torchvision.transforms.RandomErasing(p=0.5, scale=(0.02, 0.25), ratio=(0.3, 3.3))])

In [ ]:
batch_size = 32
    
transform_train = torchvision.transforms.Compose([
  torchvision.transforms.RandomCrop(32, padding=4),
  torchvision.transforms.RandomHorizontalFlip(),
  torchvision.transforms.RandomResizedCrop(32, scale=(0.8, 1.0), ratio=(0.8, 1.2)),
  torchvision.transforms.ToTensor(),
  torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

transform_test = torchvision.transforms.Compose([
                  torchvision.transforms.ToTensor(), 
                  torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])

trainset = torchvision.datasets.CIFAR10(root = './data', train = True, download = True, transform = transform_train)

# Split the train data into train and validation sets
train_size = int(0.8 * len(trainset))
val_size = len(trainset) - train_size
trainset, valset = torch.utils.data.random_split(trainset, [train_size, val_size])

# train_size = int(0.2 * len(trainset))
# val_size = int(0.2 * len(valset))
# trainset, _ = torch.utils.data.random_split(trainset, [train_size, len(trainset) - train_size])
# valset, _ = torch.utils.data.random_split(valset, [val_size, len(valset) - val_size])

testset = torchvision.datasets.CIFAR10(root = './data', train = False, download = True, transform = transform_test)

train_loader = torch.utils.data.DataLoader(trainset, batch_size = batch_size, shuffle = True)
val_loader   = torch.utils.data.DataLoader(valset, batch_size = batch_size, shuffle = True)
test_loader  = torch.utils.data.DataLoader(testset, batch_size = batch_size, shuffle = False)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
    
best_test_acc = 0

torch.cuda.empty_cache()
model = ZigZag_ResNet(BasicBlock, [2, 2, 2, 2, 2, 1, 1])
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Num Params: {num_params}\n")
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), 0.01, momentum = 0.8, weight_decay = 0.0005 , nesterov=True)

scheduler = ZigZagLROnPlateauRestarts(optimizer, mode='max', lr=0.01,
                                      up_factor=0.3, down_factor=0.5, 
                                      up_patience=1, down_patience=1, 
                                      restart_after=30, verbose = True)
train_losses_ = []
train_accuracies_ = []
valid_losses_ = []
valid_accuracies_ = []

epochs = 250
for epoch in range(epochs):
    print(f"\n\tEpoch: {epoch}")

    train_loss, train_accuracy, val_loss, val_accuracy = train_val(model, criterion, optimizer, 
                                                                train_loader, val_loader, device,
                                                                scheduler = scheduler, use_scheduler = True)
    train_losses_.append(train_loss)
    train_accuracies_.append(train_accuracy)
    valid_losses_.append(val_loss)
    valid_accuracies_.append(val_accuracy)
    print(f"\tTraining Loss: {round(train_loss, 4)}; Training Accuracy: {round(train_accuracy*100, 4)}%")
    print(f"\tValidation Loss: {round(val_loss, 4)}; Validation Accuracy: {round(val_accuracy*100, 4)}%")

test_loss, test_accuracy = test(model, criterion, test_loader, device)
print(f"\n\tTesting Loss: {round(test_loss, 4)}; Testing Accuracy: {round(test_accuracy*100, 4)}%")

if test_accuracy > best_test_acc:
    best_test_acc = test_accuracy

    metrics_dict = {'train_loss': train_losses_, 'train_accuracy': train_accuracies_, 
                  'valid_loss': valid_losses_, 'valid_accuracy': valid_accuracies_,
                  'test_loss': test_loss, 'test_accuracy': test_accuracy}

Files already downloaded and verified
Files already downloaded and verified
Num Params: 4891338


	Epoch: 0
	Training Loss: 3.4899; Training Accuracy: 35.27%
	Validation Loss: 1.5092; Validation Accuracy: 44.51%

	Epoch: 1
increasing learning rate of group 0 to 1.3000e-02.
	Training Loss: 2.7668; Training Accuracy: 50.0025%
	Validation Loss: 1.2261; Validation Accuracy: 56.45%

	Epoch: 2
	Training Loss: 2.3352; Training Accuracy: 58.475%
	Validation Loss: 0.9845; Validation Accuracy: 64.66%

	Epoch: 3
increasing learning rate of group 0 to 1.6900e-02.
	Training Loss: 1.9294; Training Accuracy: 66.245%
	Validation Loss: 0.9166; Validation Accuracy: 67.94%

	Epoch: 4
	Training Loss: 1.7819; Training Accuracy: 69.1175%
	Validation Loss: 0.9183; Validation Accuracy: 68.11%

	Epoch: 5
increasing learning rate of group 0 to 2.1970e-02.
	Training Loss: 1.5816; Training Accuracy: 73.2125%
	Validation Loss: 0.773; Validation Accuracy: 73.28%

	Epoch: 6
	Training Loss: 1.5324; Training Accuracy:

In [39]:
import numpy as np

class ZigZagLROnPlateauRestarts(torch.optim.lr_scheduler._LRScheduler):
    def __init__(self, optimizer, mode='min', lr=0.01, up_factor=1.1, down_factor=0.8, up_patience=10, down_patience=10, restart_after=30, verbose=True):
        super(ZigZagLROnPlateauRestarts).__init__()
        self.optimizer = optimizer
        self.mode = mode
        self.up_factor = 1 + up_factor
        self.down_factor = 1 - down_factor
        self.up_patience = up_patience
        self.down_patience = down_patience
        self.num_bad_epochs = 0
        self.num_good_epochs = 0
        self.prev_metric = np.Inf if self.mode == 'min' else -np.Inf
        self.best_lr = lr
        self.restart_after = restart_after
        self.verbose = verbose
        self.num_epochs = 0

    def step(self, metric):
        self.num_epochs += 1
        if self.mode == 'min':
            if metric < self.prev_metric:
                self.best_lr = self.optimizer.param_groups[0]['lr']
                self.num_bad_epochs = 0
                self.num_good_epochs += 1
                if self.num_good_epochs > self.up_patience:
                    old_lr = self.optimizer.param_groups[0]['lr']
                    new_lr = old_lr * self.up_factor
                    self.optimizer.param_groups[0]['lr'] = new_lr
                    if self.verbose:
                        print(f"increasing learning rate of group 0 to {new_lr:.4e}.")
                    self.num_good_epochs = 0
            else:
                self.num_bad_epochs += 1
                self.num_good_epochs = 0
                if self.num_bad_epochs > self.down_patience:
                    old_lr = self.optimizer.param_groups[0]['lr']
                    new_lr = old_lr * self.down_factor
                    self.optimizer.param_groups[0]['lr'] = new_lr
                    if self.verbose:
                        print(f"reducing learning rate of group 0 to {new_lr:.4e}.")
                    self.num_bad_epochs = 0
        else:
            if metric > self.prev_metric:
                self.best_lr = self.optimizer.param_groups[0]['lr']
                self.num_bad_epochs = 0
                self.num_good_epochs += 1
                if self.num_good_epochs > self.up_patience:
                    old_lr = self.optimizer.param_groups[0]['lr']
                    new_lr = old_lr * self.up_factor
                    self.optimizer.param_groups[0]['lr'] = new_lr
                    if self.verbose:
                        print(f"increasing learning rate of group 0 to {new_lr:.4e}.")
                    self.num_good_epochs = 0
            else:
                self.num_bad_epochs += 1
                self.num_good_epochs = 0
                if self.num_bad_epochs > self.down_patience:
                    old_lr = self.optimizer.param_groups[0]['lr']
                    new_lr = old_lr * self.down_factor
                    self.optimizer.param_groups[0]['lr'] = new_lr
                    if self.verbose:
                        print(f"reducing learning rate of group 0 to {new_lr:.4e}.")
                    self.num_bad_epochs = 0
        self.prev_metric = metric
                    
        if self.num_epochs % self.restart_after == 0:
            self.optimizer.param_groups[0]['lr'] = self.best_lr
            if self.verbose:
                print(f"restart: setting learning rate of group 0 to best learning rate value: {self.best_lr:.4e}.")

In [40]:
def train_val(model, criterion, optimizer, train_loader, val_loader, device, scheduler = None, use_scheduler = True):
    model.train()
    train_loss = 0
    correct = 0
    count = 0
    total = 0
    for i, data in enumerate(train_loader, 0):
        image, label = data
        image = image.to(device)
        label = label.to(device)
    
        optimizer.zero_grad()
        output = model(image)
        loss = criterion(output, label)

        train_loss += loss.item()

        pred = torch.max(output.data, 1)[1]
        cur_correct = (pred == label).sum().item()
        cur_loss = loss.item()

        loss.backward()

        optimizer.step()
        
        total += label.size(0)
        correct += cur_correct
        train_loss += cur_loss

    train_accuracy = correct/total
    train_loss = train_loss/len(train_loader)
    
    model.eval()
    valid_loss = 0
    correct = 0
    count = 0
    total = 0
    for i, data in enumerate(val_loader, 0):
        image, label = data
        image = image.to(device)
        label = label.to(device)

        output = model(image)
        loss = criterion(output, label)

        pred = torch.max(output.data, 1)[1]
        cur_correct = (pred == label).sum().item()
        cur_loss = loss.item()

        total += label.size(0)
        correct += cur_correct
        valid_loss += cur_loss

    valid_accuracy = correct/total
    valid_loss = valid_loss/len(val_loader)

    val_metric = 2*valid_accuracy * (1 - valid_loss) / (valid_accuracy + (1 - valid_loss))
    train_metric = 2*train_accuracy * (1 - train_loss) / (train_accuracy + (1 - train_loss))
    val_importance = 0.7
    metric = val_importance*val_metric + (1 - val_importance)*train_metric
    
    if use_scheduler:
        scheduler.step(metric)

    return train_loss, train_accuracy, valid_loss, valid_accuracy

def test(model, criterion, dataloader, device):
    model.eval()
    test_loss = 0
    correct = 0
    count = 0
    total = 0
    for i, data in enumerate(dataloader, 0):
        image, label = data
        image = image.to(device)
        label = label.to(device)
                
        output = model(image)
        loss = criterion(output, label)

        pred = torch.max(output.data, 1)[1]
        cur_correct = (pred == label).sum().item()
        cur_loss = loss.item()
            
        total += label.size(0)
        correct += cur_correct
        test_loss += cur_loss

    accuracy = correct/total
    test_loss = test_loss/len(dataloader)

    return test_loss, accuracy

In [41]:
batch_size = 32
    
transform_train = torchvision.transforms.Compose([
  torchvision.transforms.RandomCrop(32, padding=4),
  torchvision.transforms.RandomHorizontalFlip(),
  torchvision.transforms.RandomResizedCrop(32, scale=(0.8, 1.0), ratio=(0.8, 1.2)),
  torchvision.transforms.ToTensor(),
  torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

transform_test = torchvision.transforms.Compose([
                  torchvision.transforms.ToTensor(), 
                  torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])

trainset = torchvision.datasets.CIFAR10(root = './data', train = True, download = True)

# Split the train data into train and validation sets
train_size = int(0.9 * len(trainset))
val_size = len(trainset) - train_size
trainset, valset = torch.utils.data.random_split(trainset, [train_size, val_size])

trainset.dataset.transform = transform_train
valset.dataset.transform = transform_test

testset = torchvision.datasets.CIFAR10(root = './data', train = False, download = True, transform = transform_test)

train_loader = torch.utils.data.DataLoader(trainset, batch_size = batch_size, shuffle = True)
val_loader   = torch.utils.data.DataLoader(valset, batch_size = batch_size, shuffle = True)
test_loader  = torch.utils.data.DataLoader(testset, batch_size = batch_size, shuffle = False)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
    
best_test_acc = 0

torch.cuda.empty_cache()
model = ZigZag_ResNet(BasicBlock, [2, 2, 2, 2, 2, 1, 1])
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Num Params: {num_params}\n")
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), 0.01, momentum = 0.8, weight_decay = 0.0005 , nesterov=True)

scheduler = ZigZagLROnPlateauRestarts(optimizer, mode='max', lr=0.01,
                                      up_factor=0.3, down_factor=0.5, 
                                      up_patience=1, down_patience=1, 
                                      restart_after=30, verbose = True)
train_losses_ = []
train_accuracies_ = []
valid_losses_ = []
valid_accuracies_ = []

epochs = 100
for epoch in range(epochs):
    print(f"\n\tEpoch: {epoch}")

    train_loss, train_accuracy, val_loss, val_accuracy = train_val(model, criterion, optimizer, 
                                                                train_loader, val_loader, device,
                                                                scheduler = scheduler, use_scheduler = True)
    train_losses_.append(train_loss)
    train_accuracies_.append(train_accuracy)
    valid_losses_.append(val_loss)
    valid_accuracies_.append(val_accuracy)
    print(f"\tTraining Loss: {round(train_loss, 4)}; Training Accuracy: {round(train_accuracy*100, 4)}%")
    print(f"\tValidation Loss: {round(val_loss, 4)}; Validation Accuracy: {round(val_accuracy*100, 4)}%")

test_loss, test_accuracy = test(model, criterion, test_loader, device)
print(f"\n\tTesting Loss: {round(test_loss, 4)}; Testing Accuracy: {round(test_accuracy*100, 4)}%")

if test_accuracy > best_test_acc:
    best_test_acc = test_accuracy

    metrics_dict = {'train_loss': train_losses_, 'train_accuracy': train_accuracies_, 
                  'valid_loss': valid_losses_, 'valid_accuracy': valid_accuracies_,
                  'test_loss': test_loss, 'test_accuracy': test_accuracy}

Files already downloaded and verified
Files already downloaded and verified
Num Params: 4891338


	Epoch: 0
	Training Loss: 3.0757; Training Accuracy: 44.0067%
	Validation Loss: 1.4212; Validation Accuracy: 49.7%

	Epoch: 1
increasing learning rate of group 0 to 1.3000e-02.
	Training Loss: 2.1627; Training Accuracy: 61.86%
	Validation Loss: 0.9919; Validation Accuracy: 66.06%

	Epoch: 2
	Training Loss: 1.6972; Training Accuracy: 70.7533%
	Validation Loss: 0.7266; Validation Accuracy: 74.96%

	Epoch: 3
	Training Loss: 1.3047; Training Accuracy: 77.5489%
	Validation Loss: 0.6589; Validation Accuracy: 77.36%

	Epoch: 4
increasing learning rate of group 0 to 1.6900e-02.
	Training Loss: 1.0545; Training Accuracy: 82.0311%
	Validation Loss: 0.6461; Validation Accuracy: 77.68%

	Epoch: 5
	Training Loss: 0.9977; Training Accuracy: 83.0089%
	Validation Loss: 0.6354; Validation Accuracy: 78.7%

	Epoch: 6
increasing learning rate of group 0 to 2.1970e-02.
	Training Loss: 0.8067; Training Accuracy

In [42]:
import numpy as np

class ZigZagLROnPlateau(torch.optim.lr_scheduler._LRScheduler):
    def __init__(self, optimizer, mode='min', up_factor=1.1, down_factor=0.8, up_patience=10, down_patience=10, verbose=True):
        super(ZigZagLROnPlateau).__init__()
        self.optimizer = optimizer
        self.mode = mode
        self.up_factor = 1 + up_factor
        self.down_factor = 1 - down_factor
        self.up_patience = up_patience
        self.down_patience = down_patience
        self.num_bad_epochs = 0
        self.num_good_epochs = 0
        self.best_metric = np.Inf if self.mode == 'min' else -np.Inf
        self.verbose = verbose

    def step(self, metric):
        if self.mode == 'min':
            if metric < self.best_metric:
                self.best_metric = metric
                self.num_bad_epochs = 0
                self.num_good_epochs += 1
                if self.num_good_epochs > self.up_patience:
                    old_lr = self.optimizer.param_groups[0]['lr']
                    new_lr = old_lr * self.up_factor
                    self.optimizer.param_groups[0]['lr'] = new_lr
                    if self.verbose:
                        print(f"increasing learning rate of group 0 to {new_lr:.4e}.")
                    self.num_good_epochs = 0
            else:
                self.num_bad_epochs += 1
                self.num_good_epochs = 0
                if self.num_bad_epochs > self.down_patience:
                    old_lr = self.optimizer.param_groups[0]['lr']
                    new_lr = old_lr * self.down_factor
                    self.optimizer.param_groups[0]['lr'] = new_lr
                    if self.verbose:
                        print(f"reducing learning rate of group 0 to {new_lr:.4e}.")
                    self.num_bad_epochs = 0
        else:
            if metric > self.best_metric:
                self.best_metric = metric
                self.num_bad_epochs = 0
                self.num_good_epochs += 1
                if self.num_good_epochs > self.up_patience:
                    old_lr = self.optimizer.param_groups[0]['lr']
                    new_lr = old_lr * self.up_factor
                    self.optimizer.param_groups[0]['lr'] = new_lr
                    if self.verbose:
                        print(f"increasing learning rate of group 0 to {new_lr:.4e}.")
                    self.num_good_epochs = 0
            else:
                self.num_bad_epochs += 1
                self.num_good_epochs = 0
                if self.num_bad_epochs > self.down_patience:
                    old_lr = self.optimizer.param_groups[0]['lr']
                    new_lr = old_lr * self.down_factor
                    self.optimizer.param_groups[0]['lr'] = new_lr
                    if self.verbose:
                        print(f"reducing learning rate of group 0 to {new_lr:.4e}.")
                    self.num_bad_epochs = 0

In [9]:
batch_size = 32
    
transform_train = torchvision.transforms.Compose([
  torchvision.transforms.RandomCrop(32, padding=4),
  torchvision.transforms.RandomHorizontalFlip(),
  torchvision.transforms.RandomResizedCrop(32, scale=(0.8, 1.0), ratio=(0.8, 1.2)),
  torchvision.transforms.ToTensor(),
  torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

transform_test = torchvision.transforms.Compose([
                  torchvision.transforms.ToTensor(), 
                  torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])

trainset = torchvision.datasets.CIFAR10(root = './data', train = True, download = True, transform = transform_train)

# Split the train data into train and validation sets
train_size = int(0.9 * len(trainset))
val_size = len(trainset) - train_size
trainset, valset = torch.utils.data.random_split(trainset, [train_size, val_size])

testset = torchvision.datasets.CIFAR10(root = './data', train = False, download = True, transform = transform_test)

train_loader = torch.utils.data.DataLoader(trainset, batch_size = batch_size, shuffle = True)
val_loader   = torch.utils.data.DataLoader(valset, batch_size = batch_size, shuffle = True)
test_loader  = torch.utils.data.DataLoader(testset, batch_size = batch_size, shuffle = False)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
    
best_test_acc = 0

torch.cuda.empty_cache()
model = ZigZag_ResNet(BasicBlock, [2, 2, 2, 2, 2, 1, 1])
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Num Params: {num_params}\n")
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), 0.01, momentum = 0.8, weight_decay = 0.0005 , nesterov=True)

scheduler = ZigZagLROnPlateau(optimizer, mode='max',
                              up_factor=0.3, down_factor=0.5, 
                              up_patience=1, down_patience=1, 
                              verbose = True)
train_losses_ = []
train_accuracies_ = []
valid_losses_ = []
valid_accuracies_ = []

epochs = 100
for epoch in range(epochs):
    print(f"\n\tEpoch: {epoch}")

    train_loss, train_accuracy, val_loss, val_accuracy = train_val(model, criterion, optimizer, 
                                                                train_loader, val_loader, device,
                                                                scheduler = scheduler, use_scheduler = True)
    train_losses_.append(train_loss)
    train_accuracies_.append(train_accuracy)
    valid_losses_.append(val_loss)
    valid_accuracies_.append(val_accuracy)
    print(f"\tTraining Loss: {round(train_loss, 4)}; Training Accuracy: {round(train_accuracy*100, 4)}%")
    print(f"\tValidation Loss: {round(val_loss, 4)}; Validation Accuracy: {round(val_accuracy*100, 4)}%")

test_loss, test_accuracy = test(model, criterion, test_loader, device)
print(f"\n\tTesting Loss: {round(test_loss, 4)}; Testing Accuracy: {round(test_accuracy*100, 4)}%")

if test_accuracy > best_test_acc:
    best_test_acc = test_accuracy

    metrics_dict = {'train_loss': train_losses_, 'train_accuracy': train_accuracies_, 
                  'valid_loss': valid_losses_, 'valid_accuracy': valid_accuracies_,
                  'test_loss': test_loss, 'test_accuracy': test_accuracy}

Files already downloaded and verified
Files already downloaded and verified
Num Params: 4891338


	Epoch: 0
	Training Loss: 3.4786; Training Accuracy: 35.8022%
	Validation Loss: 1.4736; Validation Accuracy: 45.24%

	Epoch: 1
increasing learning rate of group 0 to 1.3000e-02.
	Training Loss: 2.7384; Training Accuracy: 50.3756%
	Validation Loss: 1.1993; Validation Accuracy: 56.5%

	Epoch: 2
	Training Loss: 2.2896; Training Accuracy: 59.54%
	Validation Loss: 1.0006; Validation Accuracy: 63.72%

	Epoch: 3
increasing learning rate of group 0 to 1.6900e-02.
	Training Loss: 1.8775; Training Accuracy: 67.2556%
	Validation Loss: 1.0133; Validation Accuracy: 65.4%

	Epoch: 4
	Training Loss: 1.7355; Training Accuracy: 70.0556%
	Validation Loss: 0.8403; Validation Accuracy: 70.66%

	Epoch: 5
increasing learning rate of group 0 to 2.1970e-02.
	Training Loss: 1.5167; Training Accuracy: 74.0933%
	Validation Loss: 0.7653; Validation Accuracy: 73.84%

	Epoch: 6
	Training Loss: 1.4665; Training Accuracy

In [36]:
import numpy as np

class ZigZagLROnPlateauRestarts(torch.optim.lr_scheduler._LRScheduler):
    def __init__(self, optimizer, mode='min', up_factor=1.1, down_factor=0.8, 
                                              up_patience=10, down_patience=10, 
                                              max_lr=0.1, min_lr=0.00001,
                                              verbose=True):
        super(ZigZagLROnPlateauRestarts).__init__()
        self.optimizer = optimizer
        self.mode = mode
        self.up_factor = 1 + up_factor
        self.down_factor = 1 - down_factor
        self.up_patience = up_patience
        self.down_patience = down_patience
        self.num_bad_epochs = 0
        self.num_good_epochs = 0
        self.prev_metric = np.Inf if self.mode == 'min' else -np.Inf
        self.best_lr = self.optimizer.param_groups[0]['lr']
        self.max_lr = max_lr
        self.min_lr = min_lr
        self.verbose = verbose

    def step(self, metric):
        if self.mode == 'min':
            if metric < self.prev_metric:
                #self.best_metric = metric
                self.best_lr = self.optimizer.param_groups[0]['lr']
                self.num_bad_epochs = 0
                self.num_good_epochs += 1
                if self.num_good_epochs > self.up_patience:
                    old_lr = self.optimizer.param_groups[0]['lr']
                    new_lr = old_lr * self.up_factor
                    if new_lr <= self.min_lr or new_lr >= self.max_lr:
                        new_lr = self.best_lr
                    self.optimizer.param_groups[0]['lr'] = new_lr
                    if self.verbose:
                        print(f"increasing learning rate of group 0 to {new_lr:.4e}.")
                    self.num_good_epochs = 0
            else:
                self.num_bad_epochs += 1
                self.num_good_epochs = 0
                if self.num_bad_epochs > self.down_patience:
                    old_lr = self.optimizer.param_groups[0]['lr']
                    new_lr = old_lr * self.down_factor
                    if new_lr <= self.min_lr or new_lr >= self.max_lr:
                        new_lr = self.best_lr
                    self.optimizer.param_groups[0]['lr'] = new_lr
                    if self.verbose:
                        print(f"reducing learning rate of group 0 to {new_lr:.4e}.")
                    self.num_bad_epochs = 0
        else:
            if metric > self.prev_metric:
                #self.best_metric = metric
                self.best_lr = self.optimizer.param_groups[0]['lr']
                self.num_bad_epochs = 0
                self.num_good_epochs += 1
                if self.num_good_epochs > self.up_patience:
                    old_lr = self.optimizer.param_groups[0]['lr']
                    new_lr = old_lr * self.up_factor
                    if new_lr <= self.min_lr or new_lr >= self.max_lr:
                        new_lr = self.best_lr
                    self.optimizer.param_groups[0]['lr'] = new_lr
                    if self.verbose:
                        print(f"increasing learning rate of group 0 to {new_lr:.4e}.")
                    self.num_good_epochs = 0
            else:
                self.num_bad_epochs += 1
                self.num_good_epochs = 0
                if self.num_bad_epochs > self.down_patience:
                    old_lr = self.optimizer.param_groups[0]['lr']
                    new_lr = old_lr * self.down_factor
                    if new_lr <= self.min_lr or new_lr >= self.max_lr:
                        new_lr = self.best_lr
                    self.optimizer.param_groups[0]['lr'] = new_lr
                    if self.verbose:
                        print(f"reducing learning rate of group 0 to {new_lr:.4e}.")
                    self.num_bad_epochs = 0
        self.prev_metric = metric

In [37]:
def train_val(model, criterion, optimizer, train_loader, val_loader, device, scheduler = None, use_scheduler = True):
    model.train()
    train_loss = 0
    correct = 0
    count = 0
    total = 0
    for i, data in enumerate(train_loader, 0):
        image, label = data
        image = image.to(device)
        label = label.to(device)
    
        optimizer.zero_grad()
        output = model(image)
        loss = criterion(output, label)

        train_loss += loss.item()

        pred = torch.max(output.data, 1)[1]
        cur_correct = (pred == label).sum().item()
        cur_loss = loss.item()

        loss.backward()

        optimizer.step()
        
        total += label.size(0)
        correct += cur_correct
        train_loss += cur_loss

    train_accuracy = correct/total
    train_loss = train_loss/len(train_loader)
    
    model.eval()
    valid_loss = 0
    correct = 0
    count = 0
    total = 0
    for i, data in enumerate(val_loader, 0):
        image, label = data
        image = image.to(device)
        label = label.to(device)

        output = model(image)
        loss = criterion(output, label)

        pred = torch.max(output.data, 1)[1]
        cur_correct = (pred == label).sum().item()
        cur_loss = loss.item()

        total += label.size(0)
        correct += cur_correct
        valid_loss += cur_loss

    valid_accuracy = correct/total
    valid_loss = valid_loss/len(val_loader)

    val_loss_sigmoid = torch.sigmoid(torch.tensor(valid_loss))
    train_loss_sigmoid = torch.sigmoid(torch.tensor(train_loss))

    val_metric = valid_accuracy * (1 - val_loss_sigmoid)
    train_metric = train_accuracy * (1 - train_loss_sigmoid)
    val_importance = 0.5
    metric = val_importance * val_metric + (1 - val_importance) * (1- train_metric)

    print(f"\tmetric: {round(metric.item(), 4)}")
    if use_scheduler:
        scheduler.step(metric)

    return train_loss, train_accuracy, valid_loss, valid_accuracy

def test(model, criterion, dataloader, device):
    model.eval()
    test_loss = 0
    correct = 0
    count = 0
    total = 0
    for i, data in enumerate(dataloader, 0):
        image, label = data
        image = image.to(device)
        label = label.to(device)
                
        output = model(image)
        loss = criterion(output, label)

        pred = torch.max(output.data, 1)[1]
        cur_correct = (pred == label).sum().item()
        cur_loss = loss.item()
            
        total += label.size(0)
        correct += cur_correct
        test_loss += cur_loss

    accuracy = correct/total
    test_loss = test_loss/len(dataloader)

    return test_loss, accuracy

In [38]:
batch_size = 32
    
transform_train = torchvision.transforms.Compose([
  torchvision.transforms.RandomCrop(32, padding=4),
  torchvision.transforms.RandomHorizontalFlip(),
  torchvision.transforms.RandomResizedCrop(32, scale=(0.8, 1.0), ratio=(0.8, 1.2)),
  torchvision.transforms.ToTensor(),
  torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

transform_test = torchvision.transforms.Compose([
                  torchvision.transforms.ToTensor(), 
                  torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])

trainset = torchvision.datasets.CIFAR10(root = './data', train = True, download = True, transform = transform_train)

# Split the train data into train and validation sets
train_size = int(0.9 * len(trainset))
val_size = len(trainset) - train_size
trainset, valset = torch.utils.data.random_split(trainset, [train_size, val_size])

testset = torchvision.datasets.CIFAR10(root = './data', train = False, download = True, transform = transform_test)

train_loader = torch.utils.data.DataLoader(trainset, batch_size = batch_size, shuffle = True)
val_loader   = torch.utils.data.DataLoader(valset, batch_size = batch_size, shuffle = True)
test_loader  = torch.utils.data.DataLoader(testset, batch_size = batch_size, shuffle = False)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
    
best_test_acc = 0

torch.cuda.empty_cache()
model = ZigZag_ResNet(BasicBlock, [2, 2, 2, 2, 2, 1, 1])
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Num Params: {num_params}\n")
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), 0.01, momentum = 0.8, weight_decay = 0.0005 , nesterov=True)

scheduler = ZigZagLROnPlateauRestarts(optimizer, mode='max',
                                      up_factor=0.3, down_factor=0.5, 
                                      up_patience=1, down_patience=1, 
                                      max_lr=0.1, min_lr=0.00001,
                                      verbose = True)
train_losses_ = []
train_accuracies_ = []
valid_losses_ = []
valid_accuracies_ = []

epochs = 100
for epoch in range(epochs):
    print(f"\n\tEpoch: {epoch}")

    train_loss, train_accuracy, val_loss, val_accuracy = train_val(model, criterion, optimizer, 
                                                                train_loader, val_loader, device,
                                                                scheduler = scheduler, use_scheduler = True)
    train_losses_.append(train_loss)
    train_accuracies_.append(train_accuracy)
    valid_losses_.append(val_loss)
    valid_accuracies_.append(val_accuracy)
    print(f"\tTraining Loss: {round(train_loss, 4)}; Training Accuracy: {round(train_accuracy*100, 4)}%")
    print(f"\tValidation Loss: {round(val_loss, 4)}; Validation Accuracy: {round(val_accuracy*100, 4)}%")

test_loss, test_accuracy = test(model, criterion, test_loader, device)
print(f"\n\tTesting Loss: {round(test_loss, 4)}; Testing Accuracy: {round(test_accuracy*100, 4)}%")

if test_accuracy > best_test_acc:
    best_test_acc = test_accuracy

    metrics_dict = {'train_loss': train_losses_, 'train_accuracy': train_accuracies_, 
                  'valid_loss': valid_losses_, 'valid_accuracy': valid_accuracies_,
                  'test_loss': test_loss, 'test_accuracy': test_accuracy}

Files already downloaded and verified
Files already downloaded and verified
Num Params: 4891338


	Epoch: 0
	metric: 0.516
	Training Loss: 3.4402; Training Accuracy: 36.6133%
	Validation Loss: 1.94; Validation Accuracy: 34.54%

	Epoch: 1
	metric: 0.5387
increasing learning rate of group 0 to 1.3000e-02.
	Training Loss: 2.6793; Training Accuracy: 51.6444%
	Validation Loss: 1.3224; Validation Accuracy: 52.58%

	Epoch: 2
	metric: 0.5332
	Training Loss: 2.2076; Training Accuracy: 60.8933%
	Validation Loss: 1.2521; Validation Accuracy: 56.96%

	Epoch: 3
	metric: 0.5583
	Training Loss: 1.8518; Training Accuracy: 67.6711%
	Validation Loss: 0.8671; Validation Accuracy: 70.44%

	Epoch: 4
	metric: 0.5582
	Training Loss: 1.6209; Training Accuracy: 72.0022%
	Validation Loss: 0.7524; Validation Accuracy: 73.44%

	Epoch: 5
	metric: 0.5508
reducing learning rate of group 0 to 6.5000e-03.
	Training Loss: 1.4461; Training Accuracy: 75.2311%
	Validation Loss: 0.7249; Validation Accuracy: 75.06%

	Epoch:

In [17]:
epochs = 100
for epoch in range(epochs):
    print(f"\n\tEpoch: {epoch + 100}")

    train_loss, train_accuracy, val_loss, val_accuracy = train_val(model, criterion, optimizer, 
                                                                train_loader, val_loader, device,
                                                                scheduler = scheduler, use_scheduler = True)
    train_losses_.append(train_loss)
    train_accuracies_.append(train_accuracy)
    valid_losses_.append(val_loss)
    valid_accuracies_.append(val_accuracy)
    print(f"\tTraining Loss: {round(train_loss, 4)}; Training Accuracy: {round(train_accuracy*100, 4)}%")
    print(f"\tValidation Loss: {round(val_loss, 4)}; Validation Accuracy: {round(val_accuracy*100, 4)}%")

test_loss, test_accuracy = test(model, criterion, test_loader, device)
print(f"\n\tTesting Loss: {round(test_loss, 4)}; Testing Accuracy: {round(test_accuracy*100, 4)}%")

if test_accuracy > best_test_acc:
    best_test_acc = test_accuracy

    metrics_dict = {'train_loss': train_losses_, 'train_accuracy': train_accuracies_, 
                  'valid_loss': valid_losses_, 'valid_accuracy': valid_accuracies_,
                  'test_loss': test_loss, 'test_accuracy': test_accuracy}


	Epoch: 100
	Training Loss: 0.157; Training Accuracy: 97.3556%
	Validation Loss: 0.2582; Validation Accuracy: 91.84%

	Epoch: 101
reducing learning rate of group 0 to 4.0625e-04.
	Training Loss: 0.1477; Training Accuracy: 97.5133%
	Validation Loss: 0.2515; Validation Accuracy: 92.36%

	Epoch: 102
	Training Loss: 0.1343; Training Accuracy: 97.7467%
	Validation Loss: 0.2398; Validation Accuracy: 92.44%

	Epoch: 103
reducing learning rate of group 0 to 2.0313e-04.
	Training Loss: 0.1302; Training Accuracy: 97.8378%
	Validation Loss: 0.2473; Validation Accuracy: 92.26%

	Epoch: 104
	Training Loss: 0.1235; Training Accuracy: 97.9489%
	Validation Loss: 0.2448; Validation Accuracy: 92.42%

	Epoch: 105
reducing learning rate of group 0 to 1.0156e-04.
	Training Loss: 0.1192; Training Accuracy: 97.98%
	Validation Loss: 0.2453; Validation Accuracy: 92.1%

	Epoch: 106
	Training Loss: 0.1171; Training Accuracy: 98.1067%
	Validation Loss: 0.2531; Validation Accuracy: 92.34%

	Epoch: 107
reducing le

KeyboardInterrupt: 